<a href="https://colab.research.google.com/github/ChavChavC/BT4222/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# Upload a CSV file from your local machine
uploaded = files.upload()

Saving train-00000-of-00001-f6c9e78174b9882e.parquet to train-00000-of-00001-f6c9e78174b9882e.parquet
Saving test-00000-of-00001-f8c02546a74f96bb.parquet to test-00000-of-00001-f8c02546a74f96bb.parquet


# Data Cleaning

In [ ]:
import pandas as pd
import re

test_1 = pd.read_parquet('test-00000-of-00001-f8c02546a74f96bb.parquet')
train_1 = pd.read_parquet('train-00000-of-00001-f6c9e78174b9882e.parquet')

#concatenante second with first
#labels are: 0: Negative, 1: Positive , 2: Neutral
df = pd.concat([test_1, train_1],ignore_index=True)

#0: Negative, 1: Positive , 2: Neutral
test_2 = pd.read_csv('sent_train.csv')
train_2 = pd.read_csv('sent_valid.csv')

df_2 = pd.concat([test_2, train_2], ignore_index=True)

#rename the values in df_2['label'] to 0: Negative, 1: Positive , 2: Neutral
# df_2.loc[df_2['label'] == 1, 'label'] = 2
# df_2.loc[df_2['label'] == 2, 'label'] = 1
# df_2 = df_2.rename(columns={'text':'title', 'label':'labels'})
# #df_2 concat with df
# data = pd.concat([df[['title', 'labels']], df_2[['title', 'labels']]], ignore_index=True)


#rename the values in df_2['label'] to 0: Negative, 1: Positive , 2: Neutral
new_df_2 = pd.DataFrame()
df_2_sentiment0 = df_2.loc[df_2['label']==0]

#rename the values in df_2['label'] = 1 to 2
df_2_sentiment1 = df_2.loc[df_2['label']==1]
df_2_sentiment1.loc[df_2_sentiment1['label'] == 1, 'label'] = 2

#rename the values in df_2['label'] = 2 to 1
df_2_sentiment2 = df_2.loc[df_2['label']==2]
df_2_sentiment2.loc[df_2_sentiment2['label'] == 2, 'label'] = 1

new_df_2 = pd.concat([df_2_sentiment0, df_2_sentiment1, df_2_sentiment2],ignore_index=True)
new_df_2 = new_df_2.rename(columns={'text':'title', 'label':'labels'})

#df_2 concat with df
data = pd.concat([df[['title', 'labels']], new_df_2[['title', 'labels']]], ignore_index=True)

In [ ]:
#Regex cleaning to remove URLs
data['title'] = data['title'].replace(to_replace=r'https?:\/\/.*[\r\n]*',value='',regex=True)

In [ ]:
#regex remove hashtags, newline and twitter usernames (@)
data['title'] = data['title'].replace(to_replace=r'#',value='',regex=True)
data['title'] = data['title'].replace(to_replace=r'\n',value='',regex=True)
data['title'] = data['title'].replace(to_replace=r'@[^\s]+',value='',regex=True)

In [ ]:
#remove symbols
symbol_pattern = r'[!@#%^&*()_+{}\[\]:;"\'<>,.?\\|\\/`~0-9]'
data['title'] = data['title'].str.replace(symbol_pattern, '', regex=True)

In [ ]:
data.replace("", float("NaN"), inplace=True)
data.dropna()

,title,labels
0,Gildan Activewear Reports Strong Results for t...,2
1,TRILLION ENERGY ANNOUNCES FLOW TEST RESULTS FO...,2
2,CAPREIT Announces October Distribution,1
3,Unigold Inc Delivers Positive Feasibility Stud...,2
4,Wallbridge Provides Update on Archer Explorati...,1
...,...,...
13705,Stocks making the biggest moves midday TD Amer...,1
13706,Stocks making the biggest moves premarket Fitb...,1
13707,Stocks making the biggest moves premarket Home...,1
13708,Stocks making the biggest moves premarket TD A...,1


In [ ]:
import nltk
from nltk.corpus import words

nltk.download("words")
nltk.download('punkt')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Download the English words corpus
nltk.download("words")

# Create a function to check if a word is in the English vocabulary
english_vocab = set(w.lower() for w in words.words())

def is_english_word(word):
    return word.lower() in english_vocab

# Create a function to remove non-English words from a text
def remove_words_with_non_english_characters(text):
    if isinstance(text, str):
        words = text.split()  # Split the text into words for strings
        cleaned_words = []
        for word in words:
            if all(char.isalpha() and ord(char) < 128 for char in word):
                cleaned_words.append(word)
            else:
                cleaned_words.append(word)  # Retain symbols and punctuation
        cleaned_text = ' '.join(cleaned_words)
        return cleaned_text
    else:
        return text


new_data = data.copy()
new_data["title"] = new_data["title"].apply(remove_words_with_non_english_characters)

# df["text"] now contains text with non-English words removed, while other columns remain intact
print(new_data)

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


                                                   title  labels
0      Gildan Activewear Reports Strong Results for t...       2
1      TRILLION ENERGY ANNOUNCES FLOW TEST RESULTS FO...       2
2                 CAPREIT Announces October Distribution       1
3      Unigold Inc Delivers Positive Feasibility Stud...       2
4      Wallbridge Provides Update on Archer Explorati...       1
...                                                  ...     ...
13705  Stocks making the biggest moves midday TD Amer...       1
13706  Stocks making the biggest moves premarket Fitb...       1
13707  Stocks making the biggest moves premarket Home...       1
13708  Stocks making the biggest moves premarket TD A...       1
13709           TCO NNVC GPOR and JE among midday movers       1

[13710 rows x 2 columns]


In [ ]:
print(len(new_data))
new_data.replace("", float("NaN"), inplace=True)
cleaned_data = new_data.dropna()
cleaned_data.to_csv('cleaned_data.csv')


13710


# Lemmatization

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

### Thought process:
While lemmatization is considered computationally more expensive than stemming due to its reliance on a linguistic database, our team believes that lemmatization is superior because it yields complete English words, whereas stemming can result in incomplete or non-English words.

In [ ]:
# Import necessary libraries
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd

# Create a WordNetLemmatizer object for lemmatization
lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(text):
    """
    Lemmatizes a given text by tokenizing it into words,
    lemmatizing each word, and then rejoining them into a sentence.

    Parameters:
    - text (str): The input text to be lemmatized.

    Returns:
    - lemmatized_sentence (str): The lemmatized sentence.
    """
    # Tokenize the sentence into words
    words = word_tokenize(text)

    # Lemmatize each word and convert to lowercase
    lemmatized_words = [lemmatizer.lemmatize(word.lower()) for word in words]

    # Join the lemmatized words back into a sentence
    lemmatized_sentence = ' '.join(lemmatized_words)

    return lemmatized_sentence

def lemmatize_dataset(sentences):
    """
    Lemmatizes an entire dataset of sentences using the lemmatize_sentence function.

    Parameters:
    - sentences (pd.Series): A Pandas Series containing the sentences to be lemmatized.

    Returns:
    - lemmatized (list) : A list of lemmatized sentences.
    """
    # Initialize an empty list to store lemmatized sentences
    lemmatized = []

    # Iterate through each sentence in the dataset
    for sentence in sentences:
        # Lemmatize the current sentence and append to the list
        lemmatized.append(lemmatize_sentence(sentence))

    return lemmatized

# Lemmatize the 'title' column in the 'data' DataFrame
data["title"] = pd.Series(lemmatize_dataset(data["title"]))

data.head()
